In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import plotly.express as px
import scipy
from scipy import stats

In [ ]:
data = pd.read_excel('dataset.xlsx')
data = data.drop('активация_кэшбэка_аренда_авто', axis=1).drop('кэшбэк_аренда_авто', axis=1).drop('оборот_аренда_авто', axis=1)

In [ ]:
data['месяц_покупок'] = data['месяц_покупок'].astype(str)
for i in range(4, 10):
    d = datetime.strptime(f'2023-0{i}-01', "%Y-%m-%d")
    data['месяц_покупок'] = data['месяц_покупок'].replace(f'2023-0{i}-01', d.strftime("%B"))
data

In [ ]:
categories = 'аптеки рестораны одежда_и_обувь автоуслуги супермаркеты такси красота развлечения жд_билеты образование дом_и_ремонт спорттовары животные цветы фастфуд каршеринг'.split()
turnover = []
cashback_active = []
cashback = []
for i in categories:
    turnover.append('оборот_' + i)
    cashback_active.append('активация_кэшбэка_' + i)
    cashback.append('кэшбэк_' + i)

In [ ]:
data

# 1 гипотеза v1.0

In [ ]:
mean_turnover = data.groupby('ключ_клиента', as_index=False).aggregate({i: 'mean' for i in turnover})
mean_turnover = mean_turnover.fillna(0)

In [ ]:
mean_turnover

In [ ]:
cashback_table = data[['ключ_клиента'] + cashback_active].fillna(0).astype(int).to_numpy()

In [ ]:
turnover_table = data[turnover].fillna(0).astype(int).to_numpy()
turnover_table = np.hstack([np.ones(len(turnover_table), dtype=int).reshape((len(turnover_table), 1)), turnover_table])

In [ ]:
columns = ['ключ_клиента']+turnover
spending_with_cashback = turnover_table*cashback_table
spending_with_cashback = {columns[i]: spending_with_cashback.T[i] for i in range(len(columns))}
spending_with_cashback = pd.DataFrame(data=spending_with_cashback)

In [ ]:
s = []
for row in spending_with_cashback.iloc:
    person_mean = mean_turnover[mean_turnover['ключ_клиента'] == row['ключ_клиента']].to_numpy()[0][1:]
    row = row.to_numpy()[1:]
    k = []
    for i in range(len(row)):
        if row[i] != 0:
            k.append(row[i] - person_mean[i])
    s = s + k

In [ ]:
s = list(filter(lambda i: i != 0,s))

In [ ]:
np.array(s).mean()

# 1 гипотеза v2.0

In [ ]:
# Полность данных по обороту, где желтый - пропуски(NaN), синий - наличие данных (не NaN)
colours = ['#000099', '#ffff00']
plt.figure(figsize=(10,10), dpi=200)
sns.heatmap(data[turnover].isnull(), cmap=sns.color_palette(colours))

In [ ]:
def clean_data(X):
    X = X[~np.isnan(X)]
    mu = X.mean()
    om = sum((X-mu)**2) / len(X)
    sigma = om ** 0.5
    N = 3
    l, r = int(mu - N * sigma), int(mu + N * sigma)
    X0 = X[(X > l) & (X < r)]
    return X0, len(X0) / len(X)

def clean_cat(turnover_cat_name, active_cat_name):
    df_cat = data[data[turnover_cat_name].isna() == False][[turnover_cat_name, active_cat_name]]
    df_cat[turnover_cat_name] = df_cat[turnover_cat_name] ** 0.5
    mu = df_cat[turnover_cat_name].mean()
    sigma = (sum((df_cat[turnover_cat_name] - mu) ** 2) / len(df_cat)) ** 0.5
    N = 3
    l, r = int(mu - N * sigma), int(mu + N * sigma)
    df_cat = df_cat[(df_cat[turnover_cat_name] >= l) & (df_cat[turnover_cat_name] <= r)]
    return df_cat

In [ ]:
# Дисперсия в предобработтанных данных
om = lambda i: sum((i - i.mean()) ** 2) / len(i)
print('категория\tактивный\tнеактивный')
for cat in categories:
    turnover_cat_name = 'оборот_' + cat
    active_cat_name = 'активация_кэшбэка_' + cat
    df_cat = clean_cat(turnover_cat_name, active_cat_name)
    with_active_cashback = df_cat[df_cat[active_cat_name ] == 1][turnover_cat_name]
    with_inactive_cashback = df_cat[df_cat[active_cat_name] != 1][turnover_cat_name]
    print(f'{cat.ljust(14, " ")}\t{round(om(with_active_cashback), 5)}\t{round(om(with_inactive_cashback), 5)}')

In [ ]:
# Проверка гипотезы на одной категории(аптеки)
df_pharmacy = clean_cat('оборот_аптеки', 'активация_кэшбэка_аптеки')
pharmacy_active = df_pharmacy[df_pharmacy['активация_кэшбэка_аптеки'] == 1]['оборот_аптеки']
pharmacy_inactive = df_pharmacy[df_pharmacy['активация_кэшбэка_аптеки'] != 1]['оборот_аптеки']
stats.ttest_ind(pharmacy_inactive.to_list(), 
                pharmacy_active.to_list(), 
                alternative='less',
                equal_var = False
               )

In [ ]:
data_new = []
for cat in turnover:
    cat_data = data[data[cat].isna() == False][cat]
    cat_sqrt = cat_data**(1/2)
    
    name = cat.replace('_', ' ')
    name_cat = " ".join(cat.split("_")[1:])
    name = name[0].upper() + name[1:]
    
    fig = px.histogram(cat_data, width=800, height=400)
    fig.update_layout(
        xaxis_title="Оборот",
        yaxis_title="Количество",
        title_text = name + ' до обработки',
        title_font_size=20,
        title_x=0.5,
        showlegend=False,
    )
    fig.show()
    
    fig = px.histogram(cat_sqrt, width=800, height=400) 
    fig.update_layout(
        xaxis_title="Оборот",
        yaxis_title="Количество",
        title_text = name + ' после нормализации',
        title_font_size=20,
        title_x=0.5,
        showlegend=False,
    )
    fig.show()
    
    сlean, l = clean_data(cat_sqrt.values)
    
    fig = px.histogram(сlean, width=800, height=400)

    fig.update_layout(
        xaxis_title="Оборот",
        yaxis_title="Количество",
        title_text = name + ' после отчистки выбросов',
        title_font_size=20,
        title_x=0.5,
        showlegend=False,
    )
    fig.show()
    
    data_new.append(сlean)
    fig = px.box(x=cat_data, title=f'Изначальные данные категории "{name_cat}"', width=800, height=400)

    fig.update_layout(
        title_font_size=20,
        title_x=0.5,
        showlegend=False,
    )
#     fig.update_xaxes(showgrid=False)
#     fig.update_yaxes(showgrid=False)
    fig.show()
    
    fig = px.box(x=сlean, title=f'После обработки (категория {name_cat})', width=800, height=400)
    
    fig.update_layout(
        title_font_size=20,
        title_x=0.5,
        showlegend=False,
    )
#     fig.update_xaxes(showgrid=False)
#     fig.update_yaxes(showgrid=False)
    fig.show()
    print(f'Процент данных, которые остались после отчистки по категории "{name_cat}": {l}')

# Среднее квадратическое отклонение трат

In [ ]:
df = data.groupby('месяц_покупок', as_index=False).aggregate({i: 'sum' for i in turnover})

In [ ]:
df

In [ ]:
for i in turnover:
    fig = px.bar(x = df['месяц_покупок'], y = df[i], title=i, width=800, height=400)
    fig.show()

In [ ]:
data['сумма оборота'] = sum([data[data[i].isna() == False][i] for i in turnover])
df1 = data.groupby('ключ_клиента', as_index=False)['сумма оборота'].agg(lambda x: (((x-x.mean())**2).sum()/len(x))**0.5)

In [ ]:
data

In [ ]:
df1 = df1.rename(columns= {'сумма оборота': 'RMSE оборота'})

In [ ]:
df1['фааф'] = round(df1['RMSE оборота']/((data['сумма оборота']+1e-9)/6), 6)

In [ ]:
df1 = df1.sort_values('фааф', ascending=True)

In [ ]:
df1['фааф']

In [ ]:
sns.histplot(df1['фааф'], bins=200)

In [ ]:
df1.to_excel("фааф.xlsx")  

# Оборот в городах-миллионниках

In [ ]:
data_city_million = data[(data['город_проживания'] == 'Москва') | (data['город_проживания'] == 'Санкт-Петербург') | (data['город_проживания'] == 'Екатеринбург') | (data['город_проживания'] == 'Казань') | (data['город_проживания'] == 'Новосибирск')]

In [ ]:
actives = []
for i in range(len(data_city_million)):
    actives.append(data_city_million[cashback_active].iloc[i].to_list().count(1))

In [ ]:
data_city_million[cashback_active]

In [ ]:
turnover_amount = []
for i in turnover:
    turnover_amount.append(data_city_million[i].sum())

In [ ]:
fig = plt.figure(figsize =(10, 10))
plt.pie(turnover_amount, labels = categories)
plt.show()

In [ ]:
turnover_in_cashback = []
for i in range(len(turnover)):
    turnover_in_cashback.append(data_city_million[data_city_million[cashback_active[i]] == 1][turnover[i]].sum())

In [ ]:
fig = plt.figure(figsize =(10, 10))
plt.pie(turnover_in_cashback, labels = categories)
plt.show()

In [ ]:
cashback_amount = []
for i in cashback:
    cashback_amount.append(data_city_million[i].sum())

In [ ]:
df = []
for i in range(len(categories)):
    df.append([categories[i], cashback_amount[i], turnover_in_cashback[i], turnover_amount[i]])

In [ ]:
df = pd.DataFrame(df, columns=['Category', 'cashback amount', 'turnover in cashback', 'turnover']) 
df

In [ ]:
df.plot(x = 'Category',
        kind = 'barh',
        stacked = True,
        mark_right = True,
        figsize=(15,10)) 